In [34]:
from sagemaker import get_execution_role
from sagemaker import Session

role = "arn:aws:iam::049049517134:role/service-role/AmazonSageMaker-ExecutionRole-20210901T201342"
sess = Session()
bucket = sess.default_bucket()

In [35]:
# reference: https://github.com/keras-team/keras/issues/14265
!pip install "h5py==2.10.0"
import h5py
import numpy as np

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [36]:
# Model files were manually moved to keras_model
#!mkdir keras_model
#!wget https://s3.amazonaws.com/aws-ml-blog/artifacts/keras-tensorflow-model-deployment/model.zip
#!unzip model.zip -d keras_model

In [37]:
# Load the file from directory
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json

with open(os.path.join('keras_model', 'nn_model.json'), 'r') as fp:
    loaded_model_json = fp.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('keras_model/nn_model.h5')

In [38]:
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

1.15.5


In [39]:
# Convert the keras model to Amazon protobuf

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

#if tf.executing_eagerly():
#    tf.compat.v1.disable_eager_execution()

# Note: This directory structure will need to be followed 
model_version = '1'
export_dir = 'export/Servo/' + model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

session = tf.compat.v1.Session()
init_op = tf.compat.v1.global_variables_initializer()
session.run(init_op)
# Save the meta graph and variables
builder.add_meta_graph_and_variables(
    sess=session, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
builder.save() 

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


b'export/Servo/1/saved_model.pb'

In [40]:
#Convert TensorFlow model to an Amazon SageMaker-readable format

model_path = 'export/Servo/1/'
!saved_model_cli show --all --dir {model_path}



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: dense_200_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['score'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: dense_205/Sigmoid:0
  Method name is: tensorflow/serving/predict


In [41]:
# Tar the entire directory and upload to Amazon S3.
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model')

In [42]:
# deploy the model

from sagemaker.tensorflow.serving import Model
instance_type = 'ml.m4.xlarge' 

sm_model = Model(model_data=model_data, 
framework_version=tf_framework_version,role=role)


The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [43]:
%%time
uncompiled_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type) 

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!CPU times: user 130 ms, sys: 2.89 ms, total: 132 ms
Wall time: 2min 1s


In [45]:
# The sample model expects an input of shape [1,11]
data =  [0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709,0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709]
uncompiled_predictor.predict([0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709,0.35397701,  0.72085963, -0.55290383,  1.2567573 , -0.30380709])

{'predictions': [[0.463993549]]}

In [ ]:
#uncompiled_predictor.delete_endpoint()